# 加载库函数

In [1]:
import numpy as np
import scipy.sparse as sp
import networkx as nx
import networkx.algorithms.isomorphism as iso
import itertools
import os.path
import subprocess
import torch.nn.functional as F
import pickle
import os
import scipy.sparse as sp
import torch
import json
import scipy.io as sio
from torch.nn import Embedding
#import scipy.sparse as spsparse
from torch.nn import Parameter
import torch
import torch.nn as nn
from scipy.sparse import csr_matrix
import torch.nn.functional as F
import argparse
from sklearn.metrics import average_precision_score, roc_auc_score, accuracy_score, f1_score, precision_score, recall_score

# 加载数据，数据处理

In [2]:
"""
Disclaimer: functions defined from lines 15 to 36 in this file come from 
tkipf/gae original repository on Graph Autoencoders. Moreover, the
mask_test_edges_general_link_prediction function is borrowed from 
philipjackson's mask_test_edges pull request on this same repository.
"""

def sparse_to_tuple(sparse_mx):
    if not sp.isspmatrix_coo(sparse_mx):
        sparse_mx = sparse_mx.tocoo()
    coords = np.vstack((sparse_mx.row, sparse_mx.col)).transpose()
    values = sparse_mx.data
    shape = sparse_mx.shape
    return coords, values, shape

In [3]:
# Task 3 - Bidirectional Link Prediction
def mask_test_edges_bidirectional_link_prediction(adj):
    """
    Task 3: Bidirectionality Prediction: get Train and Test sets

    :param adj: complete sparse adjacency matrix of the graph
    :return: train incomplete adjacency matrix and test sets
    """

    # Remove diagonal elements
    adj = adj - sp.dia_matrix((adj.diagonal()[None, :], [0]), shape = adj.shape)
    adj.eliminate_zeros()
    val_edges, val_edges_false, test_edges, test_edges_false = None, None, None, None

    ## Unidirectional edges
    adj_sym = (adj + adj.T).sign()
    adj_tilde = (adj_sym - adj).T
    adj_tilde.eliminate_zeros()

    ## Bidirectional edges (they usually are few, so they are all in test set)
    adj_sym_ones = adj - adj_tilde
    adj_sym_ones.eliminate_zeros()

    # Positive node pairs in test set (bidirectional edges)
    test_edges, _, _ = sparse_to_tuple(adj_sym_ones)
    test_edges = test_edges[test_edges[:,1] > test_edges[:,0],:]

    # Negative node pairs in test set (unidirectional edges)
    test_edges_false, _, _ = sparse_to_tuple(adj_tilde)
    test_edges_false = test_edges_false[test_edges_false[:,0] > test_edges_false[:,1],:]
    test_edges_false = np.fliplr(test_edges_false)
    # Sampling of negative node pairs
    edges_negative_idx = np.arange(test_edges_false.shape[0])
    np.random.shuffle(edges_negative_idx)
    test_edges_false_idx = edges_negative_idx[:test_edges.shape[0]]
    test_edges_false = test_edges_false[test_edges_false_idx]

    # Get training incomplete adjacency matrix
    # 1 of the 2 directions of each bidirectional edge is masked
    adj_train = (adj - sp.triu(adj_sym_ones))
    all_train_edges = adj_train.nonzero()
    out_train_edges = [[all_train_edges[0][i],all_train_edges[1][i]] for i in range(all_train_edges[0].shape[0])]
    # Validation set: not implemented for Task 2
    # val_edges and val_edges_false are None
    return np.array(out_train_edges), val_edges_false, test_edges, test_edges_false

In [4]:
def load_data(dataset):
    """ Load datasets from text files

    :param dataset: 'cora', 'citeseer' or 'google' graph dataset.
    :return: n*n sparse adjacency matrix and n*f node-level feature matrix

    Note: in this paper, all three datasets are assumed to be featureless.
    As a consequence, the feature matrix is the identity matrix I_n.
    """
    if dataset == 'cora':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/cora.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Cora raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset == 'citeseer':
        adj = nx.adjacency_matrix(nx.read_edgelist("directed_data/citeseer.cites",
                                                   delimiter='\t',
                                                   create_using=nx.DiGraph()))
        # Transpose the adjacency matrix, as Citeseer raw dataset comes with a
        # <ID of cited paper> <ID of citing paper> edgelist format.
        adj = adj.T
        features = sp.identity(adj.shape[0])

    elif dataset in ['Wisconsin', 'Texas', 'Cornell']:
        from torch_geometric.datasets import WebKB
        from scipy import sparse

        graph_data = WebKB(root='directed_data/', name=dataset)
        data = graph_data[0]

        edges = data.edge_index
        nodes_number = data.x.shape[0]
        graph_np = np.zeros((nodes_number, nodes_number))
        for i in range(edges.shape[1]):
            graph_np[edges[0, i], edges[1, i]] = 1
    
        adj = sparse.csr_matrix(graph_np)
        
        features = sp.identity(adj.shape[0])

    else:
        raise ValueError('Undefined dataset!')

    return adj, features

In [5]:
def get_graph(DATASET='cora'):
    adj_init, features = load_data(DATASET)

    #train_edges, val_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_biased_negative_samples(adj_init)
    train_edges, val_edges_false, test_edges, test_edges_false = mask_test_edges_bidirectional_link_prediction(adj_init)
    
    all_negative_edges = np.argwhere(adj_init.toarray()==0)
    
    all_train_edges_false = np.delete(all_negative_edges, test_edges_false, axis = 0)
    
    if DATASET == 'cora':
        nodes_number = 2708
    elif DATASET == 'citeseer':
        nodes_number = 3327
    elif DATASET == 'Wisconsin':
        nodes_number = 251
    elif DATASET == 'Texas':
        nodes_number = 183
    elif DATASET == 'Cornell':
        nodes_number = 183
        
    ################## 处理测试集  #############################
    test_posi_mask = [test_edges[i][0]*nodes_number+test_edges[i][1] for i in range(len(test_edges))]
    test_false_mask = [test_edges_false[i][0]*nodes_number+test_edges_false[i][1] for i in range(len(test_edges_false))]
    test_mask = test_posi_mask + test_false_mask
    test_mask = torch.tensor(test_mask)
    test_edge_true_or_false = torch.zeros(len(test_mask))
    test_edge_true_or_false[:len(test_posi_mask)] = 1
    
    ############### 处理训练集  ################################
    negative_index = np.random.choice(range(all_train_edges_false.shape[0]), train_edges.shape[0]*2, replace=False)
    train_edges_false = all_train_edges_false[negative_index]
    train_posi_mask = [train_edges[i][0]*nodes_number+train_edges[i][1] for i in range(len(train_edges))]
    train_false_mask = [train_edges_false[i][0]*nodes_number+train_edges_false[i][1] for i in range(len(train_edges_false))]
    train_mask = train_posi_mask + train_false_mask
    train_mask = torch.tensor(train_mask)
    #train_mask = torch.tensor(train_posi_mask)
    train_edge_true_or_false = torch.zeros(len(train_mask))
    train_edge_true_or_false[:len(train_posi_mask)] = 1
    
    return train_edge_true_or_false, test_edge_true_or_false, train_mask.long(), test_mask.long(), nodes_number

In [6]:
#train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)

# 定义模型

In [7]:
def predict_acc(recons_edges, true_edges):
    predict_graph = recons_edges
    predict_edges = np.array(predict_graph)
    ap = average_precision_score(true_edges, predict_edges)
    auc = roc_auc_score(true_edges, predict_edges)
    print("AP SCORE： ", ap)
    print("AUC SCORE: ", auc)
    return ap, auc

################  更新节点内容  #######################################
class reconstruction_graph(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, context_embedding):
        super(reconstruction_graph, self).__init__()
        self.context_embedding = Parameter(context_embedding)
        
    def forward(self, importance_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(self.context_embedding[:, None]-self.context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((self.context_embedding.unsqueeze(1) - importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1
        
    
class update_nodes_embedding(nn.Module):
    def __init__(self, context_embedding):
        super(update_nodes_embedding, self).__init__()
        self.reconstruction_module = reconstruction_graph(context_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, importance_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(importance_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.context_embedding.detach(), ap, auc

####################  更新节点重要性  ######################################
class reconstruction_graph_importance(nn.Module):
    """给定类簇质心，更新节点嵌入"""
    def __init__(self, importance_embedding):
        super(reconstruction_graph_importance, self).__init__()
        self.importance_embedding = Parameter(importance_embedding)
        
    def forward(self, context_embedding):
        power = float(ALPHA + 1) / 2    # 计算幂
        
        # 计算节点之间的距离 ： 值越小越有利于边的形成
        nodes_distance = torch.norm(context_embedding[:, None]-context_embedding, dim=2, p=2)
        nodes_distance = 1.0 + nodes_distance
        
        # 计算节点之间的重要性：值越大越有利于边的形成
        norm_squared = torch.sum((context_embedding.unsqueeze(1) - self.importance_embedding)**2, 2)
        nodes_importance = N / (1.0 + norm_squared) + 1.0
        importance_prod = torch.mm(nodes_importance, nodes_importance.t())   # 分子
        importance_norm = torch.norm(nodes_importance, p=2, dim=1).unsqueeze(0)   #分母(每行的长度)
        importance_norm = importance_norm**2
        importance_similar = importance_prod.div(importance_norm.t())
        
        # 计算边的形成概率
        nodes_similar = torch.div(beta*importance_similar, nodes_distance)
        return 2*sigmoid_modle(nodes_similar)-1

class update_nodes_embedding_importance(nn.Module):
    def __init__(self, importance_embedding):
        super(update_nodes_embedding_importance, self).__init__()
        self.reconstruction_module = reconstruction_graph_importance(importance_embedding)     # 更新节点嵌入
        self.optimizer = torch.optim.SGD(params=self.reconstruction_module.parameters(), lr=LR, momentum=MOMENTUM)
        self.loss_function = torch.nn.MSELoss(reduction='sum')
        
    def forward(self, train_edges, test_edges, edge_train_mask, edge_test_mask, context_embedding):
        self.reconstruction_module.train()
        for epoch in range(EPOCH):
            self.optimizer.zero_grad()
            graph_reconstruction = self.reconstruction_module(context_embedding)
            graph_train = torch.take(graph_reconstruction, edge_train_mask)
            loss = self.loss_function(train_edges, graph_train)
            loss.backward()
            self.optimizer.step()
            print(f'Epoch: {epoch:02d}, Loss: {loss.item():.4f}')
        recons_test_edges = torch.take(graph_reconstruction, edge_test_mask).detach()
        ap, auc = predict_acc(recons_test_edges, test_edges)
        return self.reconstruction_module.importance_embedding.detach(), ap, auc

In [8]:
SEED = 1234
np.random.seed(SEED)
torch.manual_seed(SEED)

context_embedding_dim = 12
importance_embedding_dim = 48
beta = 4.8
ALPHA = 1.0
LR = 0.01
MOMENTUM = 0.9
EPOCH = 5
N = 25.0    # 目前25最佳
sigmoid_modle = torch.nn.Sigmoid()

dataset = 'Wisconsin'

In [9]:
all_best_auc = []
all_best_ap = []
for i in range(10):
    train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
    context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_context_embeddings = context_embedding.weight.detach()

    importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
    nodes_importance_embeddings = importance_embedding.weight.detach()
    
    auc_first_best_auc = 0
    auc_first_best_ap = 0
    ap_first_best_auc = 0
    ap_first_best_ap = 0

    for module_epoch in range(100):
        print("######################### 模块循环 ： %d ##########################"%module_epoch)
        update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
        nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
        update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
        nodes_importance_embeddings, ap, auc = update_nodes_importance_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
        if auc > auc_first_best_auc:
            auc_first_best_auc = auc
            auc_first_best_ap = ap
        if ap > ap_first_best_ap:
            ap_first_best_auc = auc
            ap_first_best_ap = ap
    all_best_ap.append(auc_first_best_ap)
    all_best_auc.append(auc_first_best_auc)
    
"""
train_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)
context_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_context_embeddings = context_embedding.weight.detach()

importance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入
nodes_importance_embeddings = importance_embedding.weight.detach()
    
auc_first_best_auc = 0
auc_first_best_ap = 0
ap_first_best_auc = 0
ap_first_best_ap = 0

for module_epoch in range(100):
    print("######################### 模块循环 ： %d ##########################"%module_epoch)
    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)
    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
    update_nodes_importance_module = update_nodes_embedding_importance(nodes_importance_embeddings)
    nodes_importance_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_context_embeddings)
    if auc > auc_first_best_auc:
        auc_first_best_auc = auc
        auc_first_best_ap = ap
    if ap > ap_first_best_ap:
        ap_first_best_auc = auc
        ap_first_best_ap = ap
"""

######################### 模块循环 ： 0 ##########################
Epoch: 00, Loss: 324.5631
Epoch: 01, Loss: 324.0020
Epoch: 02, Loss: 322.9600
Epoch: 03, Loss: 321.5267
Epoch: 04, Loss: 319.7929
AP SCORE：  0.5235992899971353
AUC SCORE:  0.4847980008329863
Epoch: 00, Loss: 317.8438
Epoch: 01, Loss: 317.7609
Epoch: 02, Loss: 317.6056
Epoch: 03, Loss: 317.3884
Epoch: 04, Loss: 317.1204
AP SCORE：  0.5315257363702083
AUC SCORE:  0.4993752603082049
######################### 模块循环 ： 1 ##########################
Epoch: 00, Loss: 316.8112
Epoch: 01, Loss: 316.4125
Epoch: 02, Loss: 315.6639
Epoch: 03, Loss: 314.6165
Epoch: 04, Loss: 313.3211
AP SCORE：  0.5441558361110055
AUC SCORE:  0.5226988754685548
Epoch: 00, Loss: 311.8259
Epoch: 01, Loss: 311.7727
Epoch: 02, Loss: 311.6723
Epoch: 03, Loss: 311.5305
Epoch: 04, Loss: 311.3530
AP SCORE：  0.5499002402725365
AUC SCORE:  0.5335276967930029
######################### 模块循环 ： 2 ##########################
Epoch: 00, Loss: 311.1449
Epoch: 01, Loss: 310.812

Epoch: 04, Loss: 255.5063
AP SCORE：  0.7155581051115111
AUC SCORE:  0.7126197417742607
Epoch: 00, Loss: 254.9492
Epoch: 01, Loss: 254.9112
Epoch: 02, Loss: 254.8393
Epoch: 03, Loss: 254.7376
Epoch: 04, Loss: 254.6096
AP SCORE：  0.7167811735505022
AUC SCORE:  0.7147022074135777
######################### 模块循环 ： 20 ##########################
Epoch: 00, Loss: 254.4592
Epoch: 01, Loss: 254.3246
Epoch: 02, Loss: 254.0696
Epoch: 03, Loss: 253.7079
Epoch: 04, Loss: 253.2524
AP SCORE：  0.7180368745330896
AUC SCORE:  0.7167846730528946
Epoch: 00, Loss: 252.7148
Epoch: 01, Loss: 252.6778
Epoch: 02, Loss: 252.6079
Epoch: 03, Loss: 252.5089
Epoch: 04, Loss: 252.3846
AP SCORE：  0.7187840575918485
AUC SCORE:  0.7172011661807579
######################### 模块循环 ： 21 ##########################
Epoch: 00, Loss: 252.2388
Epoch: 01, Loss: 252.1089
Epoch: 02, Loss: 251.8629
Epoch: 03, Loss: 251.5138
Epoch: 04, Loss: 251.0740
AP SCORE：  0.7233408028973879
AUC SCORE:  0.7197001249479383
Epoch: 00, Loss: 250.55

Epoch: 04, Loss: 223.2252
AP SCORE：  0.775691667256044
AUC SCORE:  0.7580174927113702
######################### 模块循环 ： 39 ##########################
Epoch: 00, Loss: 223.1603
Epoch: 01, Loss: 223.0852
Epoch: 02, Loss: 222.9428
Epoch: 03, Loss: 222.7406
Epoch: 04, Loss: 222.4853
AP SCORE：  0.7775935907044441
AUC SCORE:  0.7584339858392337
Epoch: 00, Loss: 222.1833
Epoch: 01, Loss: 222.1675
Epoch: 02, Loss: 222.1376
Epoch: 03, Loss: 222.0952
Epoch: 04, Loss: 222.0421
AP SCORE：  0.7763507858902107
AUC SCORE:  0.7584339858392336
######################### 模块循环 ： 40 ##########################
Epoch: 00, Loss: 221.9797
Epoch: 01, Loss: 221.9066
Epoch: 02, Loss: 221.7680
Epoch: 03, Loss: 221.5712
Epoch: 04, Loss: 221.3227
AP SCORE：  0.7775889095608163
AUC SCORE:  0.7600999583506872
Epoch: 00, Loss: 221.0287
Epoch: 01, Loss: 221.0135
Epoch: 02, Loss: 220.9847
Epoch: 03, Loss: 220.9440
Epoch: 04, Loss: 220.8929
AP SCORE：  0.7772405134014642
AUC SCORE:  0.7596834652228238
########################

Epoch: 04, Loss: 205.4991
AP SCORE：  0.767021768432627
AUC SCORE:  0.7684298209079551
######################### 模块循环 ： 58 ##########################
Epoch: 00, Loss: 205.4652
Epoch: 01, Loss: 205.4189
Epoch: 02, Loss: 205.3312
Epoch: 03, Loss: 205.2065
Epoch: 04, Loss: 205.0489
AP SCORE：  0.7674181773281317
AUC SCORE:  0.7692628071636818
Epoch: 00, Loss: 204.8624
Epoch: 01, Loss: 204.8541
Epoch: 02, Loss: 204.8383
Epoch: 03, Loss: 204.8161
Epoch: 04, Loss: 204.7881
AP SCORE：  0.7670677341205456
AUC SCORE:  0.7688463140358184
######################### 模块循环 ： 59 ##########################
Epoch: 00, Loss: 204.7554
Epoch: 01, Loss: 204.7101
Epoch: 02, Loss: 204.6242
Epoch: 03, Loss: 204.5020
Epoch: 04, Loss: 204.3478
AP SCORE：  0.7660473259572802
AUC SCORE:  0.768429820907955
Epoch: 00, Loss: 204.1651
Epoch: 01, Loss: 204.1571
Epoch: 02, Loss: 204.1418
Epoch: 03, Loss: 204.1203
Epoch: 04, Loss: 204.0933
AP SCORE：  0.7662354609127551
AUC SCORE:  0.7688463140358184
#########################

Epoch: 02, Loss: 193.4578
Epoch: 03, Loss: 193.4400
Epoch: 04, Loss: 193.4177
AP SCORE：  0.7682872957829591
AUC SCORE:  0.770512286547272
######################### 模块循环 ： 78 ##########################
Epoch: 00, Loss: 193.3914
Epoch: 01, Loss: 193.3577
Epoch: 02, Loss: 193.2937
Epoch: 03, Loss: 193.2026
Epoch: 04, Loss: 193.0876
AP SCORE：  0.7667465487796288
AUC SCORE:  0.7696793002915452
Epoch: 00, Loss: 192.9512
Epoch: 01, Loss: 192.9445
Epoch: 02, Loss: 192.9319
Epoch: 03, Loss: 192.9141
Epoch: 04, Loss: 192.8917
AP SCORE：  0.7672267408564595
AUC SCORE:  0.7700957934194086
######################### 模块循环 ： 79 ##########################
Epoch: 00, Loss: 192.8653
Epoch: 01, Loss: 192.8320
Epoch: 02, Loss: 192.7687
Epoch: 03, Loss: 192.6788
Epoch: 04, Loss: 192.5650
AP SCORE：  0.7668559241326254
AUC SCORE:  0.7692628071636818
Epoch: 00, Loss: 192.4302
Epoch: 01, Loss: 192.4235
Epoch: 02, Loss: 192.4109
Epoch: 03, Loss: 192.3930
Epoch: 04, Loss: 192.3705
AP SCORE：  0.7670016494139745
AUC

...

Epoch: 04, Loss: 236.6412
AP SCORE：  0.6931580174744145
AUC SCORE:  0.6859641815910037
Epoch: 00, Loss: 236.2594
Epoch: 01, Loss: 236.2413
Epoch: 02, Loss: 236.2070
Epoch: 03, Loss: 236.1584
Epoch: 04, Loss: 236.0972
AP SCORE：  0.694188193050745
AUC SCORE:  0.6872136609745938
######################### 模块循环 ： 29 ##########################
Epoch: 00, Loss: 236.0251
Epoch: 01, Loss: 235.9329
Epoch: 02, Loss: 235.7580
Epoch: 03, Loss: 235.5097
Epoch: 04, Loss: 235.1965
AP SCORE：  0.6938240889793136
AUC SCORE:  0.6876301541024573
Epoch: 00, Loss: 234.8262
Epoch: 01, Loss: 234.8085
Epoch: 02, Loss: 234.7748
Epoch: 03, Loss: 234.7271
Epoch: 04, Loss: 234.6671
AP SCORE：  0.6941561333583512
AUC SCORE:  0.6880466472303207
######################### 模块循环 ： 30 ##########################
Epoch: 00, Loss: 234.5962
Epoch: 01, Loss: 234.5067
Epoch: 02, Loss: 234.3370
Epoch: 03, Loss: 234.0960
Epoch: 04, Loss: 233.7920
AP SCORE：  0.6962676253878611
AUC SCORE:  0.6892961266139107
Epoch: 00, Loss: 233.432

AP SCORE：  0.7054026083922563
AUC SCORE:  0.7013744273219491
######################### 模块循环 ： 47 ##########################
Epoch: 00, Loss: 215.2623
Epoch: 01, Loss: 215.2044
Epoch: 02, Loss: 215.0947
Epoch: 03, Loss: 214.9387
Epoch: 04, Loss: 214.7415
AP SCORE：  0.6977225024761856
AUC SCORE:  0.7005414410662224
Epoch: 00, Loss: 214.5077
Epoch: 01, Loss: 214.4957
Epoch: 02, Loss: 214.4731
Epoch: 03, Loss: 214.4409
Epoch: 04, Loss: 214.4004
AP SCORE：  0.6976870116823294
AUC SCORE:  0.700124947938359
######################### 模块循环 ： 48 ##########################
Epoch: 00, Loss: 214.3524
Epoch: 01, Loss: 214.2955
Epoch: 02, Loss: 214.1875
Epoch: 03, Loss: 214.0339
Epoch: 04, Loss: 213.8396
AP SCORE：  0.6976870116823294
AUC SCORE:  0.700124947938359
Epoch: 00, Loss: 213.6094
Epoch: 01, Loss: 213.5974
Epoch: 02, Loss: 213.5748
Epoch: 03, Loss: 213.5425
Epoch: 04, Loss: 213.5019
AP SCORE：  0.6978703992756399
AUC SCORE:  0.7005414410662223
######################### 模块循环 ： 49 ###############

Epoch: 00, Loss: 200.4506
Epoch: 01, Loss: 200.4422
Epoch: 02, Loss: 200.4263
Epoch: 03, Loss: 200.4038
Epoch: 04, Loss: 200.3755
AP SCORE：  0.6876061106676018
AUC SCORE:  0.6817992503123699
######################### 模块循环 ： 66 ##########################
Epoch: 00, Loss: 200.3423
Epoch: 01, Loss: 200.2996
Epoch: 02, Loss: 200.2187
Epoch: 03, Loss: 200.1037
Epoch: 04, Loss: 199.9582
AP SCORE：  0.687455518550838
AUC SCORE:  0.6822157434402332
Epoch: 00, Loss: 199.7857
Epoch: 01, Loss: 199.7775
Epoch: 02, Loss: 199.7619
Epoch: 03, Loss: 199.7399
Epoch: 04, Loss: 199.7122
AP SCORE：  0.6874012286227095
AUC SCORE:  0.6822157434402332
######################### 模块循环 ： 67 ##########################
Epoch: 00, Loss: 199.6797
Epoch: 01, Loss: 199.6376
Epoch: 02, Loss: 199.5579
Epoch: 03, Loss: 199.4445
Epoch: 04, Loss: 199.3011
AP SCORE：  0.6888090271733652
AUC SCORE:  0.6826322365680966
Epoch: 00, Loss: 199.1311
Epoch: 01, Loss: 199.1230
Epoch: 02, Loss: 199.1078
Epoch: 03, Loss: 199.0863
Epoch: 

AP SCORE：  0.6919196384718876
AUC SCORE:  0.6842982090795501
Epoch: 00, Loss: 188.9590
Epoch: 01, Loss: 188.9539
Epoch: 02, Loss: 188.9444
Epoch: 03, Loss: 188.9309
Epoch: 04, Loss: 188.9140
AP SCORE：  0.6919196384718876
AUC SCORE:  0.6842982090795501
######################### 模块循环 ： 86 ##########################
Epoch: 00, Loss: 188.8943
Epoch: 01, Loss: 188.8620
Epoch: 02, Loss: 188.8007
Epoch: 03, Loss: 188.7135
Epoch: 04, Loss: 188.6032
AP SCORE：  0.6915239777390486
AUC SCORE:  0.6834652228238235
Epoch: 00, Loss: 188.4725
Epoch: 01, Loss: 188.4675
Epoch: 02, Loss: 188.4582
Epoch: 03, Loss: 188.4450
Epoch: 04, Loss: 188.4285
AP SCORE：  0.6915239777390486
AUC SCORE:  0.6834652228238235
######################### 模块循环 ： 87 ##########################
Epoch: 00, Loss: 188.4092
Epoch: 01, Loss: 188.3772
Epoch: 02, Loss: 188.3166
Epoch: 03, Loss: 188.2303
Epoch: 04, Loss: 188.1213
AP SCORE：  0.6914015079135291
AUC SCORE:  0.6830487296959599
Epoch: 00, Loss: 187.9921
Epoch: 01, Loss: 187.98

'\ntrain_edge, test_edge, train_mask, test_mask, nodes_number = get_graph(DATASET=dataset)\ncontext_embedding = Embedding(nodes_number, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入\nnodes_context_embeddings = context_embedding.weight.detach()\n\nimportance_embedding = Embedding(importance_embedding_dim, context_embedding_dim, sparse=True)      # 时刻为0时给定随机的嵌入\nnodes_importance_embeddings = importance_embedding.weight.detach()\n    \nauc_first_best_auc = 0\nauc_first_best_ap = 0\nap_first_best_auc = 0\nap_first_best_ap = 0\n\nfor module_epoch in range(100):\n    print("######################### 模块循环 ： %d ##########################"%module_epoch)\n    update_nodes_module = update_nodes_embedding(nodes_context_embeddings)\n    nodes_context_embeddings, ap, auc = update_nodes_module(train_edge, test_edge, train_mask, test_mask, nodes_importance_embeddings)\n    if auc > auc_first_best_auc:\n        auc_first_best_auc = auc\n        auc_first_best_ap = ap\n    if ap > ap_first_bes

In [11]:
print("MEAN AP : ", np.array(all_best_ap).mean())
print("STD AP : ", np.array(all_best_ap).std())
print("MEAN AUC : ", np.array(all_best_auc).mean())
print("STD AUC : ", np.array(all_best_auc).std())

"""
print("AUC FIRST BEST AUC: ", auc_first_best_auc)
print("AUC FIRST BEST AP: ", auc_first_best_ap)
print("AP FIRST BEST AUC: ", ap_first_best_auc)
print("AP FIRST BEST AP: ", ap_first_best_ap)
"""

MEAN AP :  0.7555399219363566
STD AP :  0.038850591554383224
MEAN AUC :  0.7504373177842566
STD AUC :  0.03816668123324394


'\nprint("AUC FIRST BEST AUC: ", auc_first_best_auc)\nprint("AUC FIRST BEST AP: ", auc_first_best_ap)\nprint("AP FIRST BEST AUC: ", ap_first_best_auc)\nprint("AP FIRST BEST AP: ", ap_first_best_ap)\n'